In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Importar entorno y familiarizarse

In [14]:
from boardgame2 import ReversiEnv
import numpy as np

# Crear 3 tipos de jugador
- Random: Selecciona uniformemente una de las acciones válidas
- Greedy: Selecciona la acción que le da más ganancia inmediata (cantidad de piezas que come). Si hay más de una acción que da máxima ganancia samplear uniformemente entre ellas
- Optimum (solo para 4x4): Usando resultados de la PI optima obtenida por policy iteration

Tener en cuenta que:
- ReversiEnv tiene los métodos get_valid y next_step y no es necesario mantener el estado del entorno
- env.PASS ([-1,  0]) es una acción valida posible y debería hacerse cuando no get_valid devuelve una matriz de ceros

Para el optimo en 4x4 bajar usar la PI obtenida en la notebook anterior guardado en /mdp

In [35]:
class GreedyPlayer():
    def __init__(self, player=1, board_shape=None, env=None, flatten_action=False):
        if (env is None) and (board_shape is None):
            print("board_shape and env can't be both None")
        if env is None:
            env = ReversiEnv(board_shape=board_shape)
        self.env = env
        self.player = player # player number. 1 o -1
        self.flatten_action = flatten_action
        self.board_shape = self.env.board.shape[0]
    
    def predict(self, board):
        # Implementar
        # Tiene que devoler la acción en la que come más piezas.
        # A igualdad de piezas comidas, samplear uniformemente
        if self.flatten_action:
            return action[0] * self.board_shape + action[1]
        else:
            return action
        
class RandomPlayer():
    def __init__(self, player=1, board_shape=None, env=None, flatten_action=False):
        if (env is None) and (board_shape is None):
            print("board_shape and env can't be both None")
        if env is None:
            env = ReversiEnv(board_shape=board_shape)
        self.env = env
        self.player = player
        self.flatten_action = flatten_action
        self.board_shape = self.env.board.shape[0]
        self.rnd = np.random.RandomState(seed=0)

    
    def predict(self, board):
        # Muestrea aleatoriamente las acciones válidas
        # Puede usar la función creada en la notebook anterior
        
            
        possible_moves = self.env.possible_moves
        ix = self.rnd.randint(0, len(possible_moves))
        action = possible_moves[ix]
        
        if self.flatten_action:
            return action[0] * self.board_shape + action[1]
        else:
            return action
        

class DictPolicyPlayer():
    def __init__(self, player=1, board_shape=4, env=None, flatten_action=False, dict_folder='pi_mdp.npy'):
        self.pi_dict = np.load(dict_folder, allow_pickle=True).item()
        if env is None:
            env = ReversiEnv(board_shape=board_shape)
        self.player = player
        self.flatten_action = flatten_action
        self.board_shape = board_shape
    
    def predict(self, board):
        # Elegir la acción optima y devolverla
        
        if self.flatten_action:
            return # Implementar
        else:
            return # Implementar

In [36]:
gp = GreedyPlayer(player=1, board_shape=4)
rp = RandomPlayer(player=1, board_shape=4)

# Verificar que el pass funciona OK

In [37]:
board = np.array([
    [-1, 0, 0, 0],
    [0, 1, 1, 0],
    [0, 1, 1, 0],
    [0, 0, 0, 0]]
)
rp.predict(board)
gp.predict(board)

AttributeError: 'ReversiEnv' object has no attribute 'possible_moves'

# Completar la función que dado dos jugadores imprima estadísticas de las partidas

In [31]:
def arena_stats(Player_1, Player_2, board_shape, N=500):
    
    env = ReversiEnv(board_shape=board_shape)
    wins_as_first = 0
    wins_as_second = 0
    plays_as_first = 0
    plays_as_second = 0
    total_steps = 0
    player_1 = Player_1(player=1, board_shape=board_shape, flatten_action=False)
    player_2 = Player_2(player=2, board_shape=board_shape, flatten_action=False)
    for i in range(N):
        # Aveces empieza un jugador, a veces el otro
        first_player = np.random.choice([-1, 1])
        player_1.player = first_player
        player_2.player = -first_player
        
        plays_as_first = plays_as_first + (first_player == 1)
        plays_as_second = plays_as_second + (first_player == -1)
        
        done = False
        n_steps = 0
        (board, player) = env.reset()
        
        while not done:
            if first_player == player:
                action = player_1.predict(env.board)
            else:
                action = player_2.predict(env.board)
            (board, player), reward, done, info = env.step(action)
            n_steps = n_steps + 1
        total_steps = total_steps + n_steps
        wins_as_first = wins_as_first + (reward == first_player) * (first_player == 1)
        wins_as_second = wins_as_second + (reward == first_player) * (first_player == -1)
    print(f'Wins as first: {wins_as_first/plays_as_first}')
    print(f'Wins as second: {wins_as_second/plays_as_second}')
    print(f'Plays as first: {plays_as_first}')
    print(f'Plays as second: {plays_as_second}')
    print(f'Avg game duration: {total_steps/N}')
        
    

In [32]:
arena_stats(DictPolicyPlayer, GreedyPlayer, 4, N=2000)

TypeError: object of type 'NoneType' has no len()

In [22]:
arena_stats(DictPolicyPlayer, RandomPlayer, 4, N=1000)

TypeError: object of type 'NoneType' has no len()

In [23]:
arena_stats(RandomPlayer, DictPolicyPlayer, 4, N=1000)

TypeError: object of type 'NoneType' has no len()

In [24]:
arena_stats(RandomPlayer, GreedyPlayer, 4, N=1000)

NameError: name 'action' is not defined

In [25]:
arena_stats(RandomPlayer, RandomPlayer, 4)

NameError: name 'action' is not defined

In [26]:
arena_stats(GreedyPlayer, GreedyPlayer, 4)

NameError: name 'action' is not defined

In [27]:
arena_stats(RandomPlayer, GreedyPlayer, 8, N=1000)

NameError: name 'action' is not defined

# Guardar todas las clases de jugadores en un player.py para que luego se puedan importar de la siguiente forma:

from players import RandomPlayer

from players import GreedyPlayer